In [ ]:
# Author: Edgar Alfonseca
# LinkedIn: https://www.linkedin.com/in/edgar-alfonseca/
# GitHub: https://github.com/edgaralfonseca
#
# This Python script batch geocodes addresses using the NYC OTI Geoclient API v2.0
#
#
# API description: https://api-portal.nyc.gov/api-details#api=geoclient-current-v2
# GitHub repo: https://github.com/mlipper/geoclient

# Pre-requisites
#
# 1) Create a new account on the NYC API Developers Portal: https://api-portal.nyc.gov/
# 2) Request an API key by subscribing to the Geoclient API in the portal


# Notes
# The OTI geoclient api handles 2,500 requests per minute / 500,000 requests per day
# NYC Department of City Planning's Geosupport (https://www.nyc.gov/site/planning/data-maps/open-data/dwn-gde-home.page) is used to power Geoclient
# Sometimes there might be a several week delay in Geoclient reflecting what is in GeoSupport
# Geoclient serves up a subset of attributes whereas Geosupport has all attributes

In [1]:
# Import necessary python modules and prepare data

import pandas as pd
import requests
import numpy as np
import time

In [4]:
# Import sample NYC address data (close to 6k records)

url = "https://raw.githubusercontent.com/edgaralfonseca/python-nyc-oti-geoclient-api-v2/refs/heads/main/nyc_sample_almost_6k_addresses.csv"

nyc_address_df = pd.read_csv(url)

# minor data cleaning on the postcode (zip code)

nyc_address_df['postcode'] = nyc_address_df['postcode'].astype(str).str[:5]

In [5]:
nyc_address_df.head(10)

,row_id,house_number,street_name,borough,postcode
0,1,114,SEIGEL STREET,Brooklyn,11206
1,2,1920,UNION STREET,Brooklyn,11233
2,3,2555,WILLIAMSBRIDGE ROAD,Bronx,10469
3,4,763,JENNINGS STREET,Bronx,10459
4,5,275,PRESIDENT STREET,Brooklyn,11231
5,6,1402,NEW YORK AVENUE,Brooklyn,11210
6,7,658,DRIGGS AVENUE,Brooklyn,11211
7,8,740,EAST 222 STREET,Bronx,10467
8,9,390,1 AVENUE,Manhattan,10010
9,10,91,VISITATION PLACE,Brooklyn,11231


**Example 1: Calling the OTI Geoclient "Address" API endpoint**

Create a custom function that takes a pandas dataframe (what you want to geocode) as an input and creates an output that is a copy of your original dataframe left joined to the API results.

In [6]:
# Create a custom function to make API calls to the 'Address' endpoint

def oti_geoclient_api_v2_address_endpoint(api_endpoint, headers, df_name, df_key_field, housenum_input_col, street_input_col, boro_input_col=None, zip_input_col=None, response_columns=None):
    """
    Fetch data from the OTI geoclient API, merge the response with the original dataframe, and return the merged dataframe.

    Parameters:
    - api_endpoint (str): The API endpoint URL.
    - headers (dict): The headers to send with the API request.
    - df_name (pd.DataFrame): The input pandas DataFrame.
    - df_key_field (str): The name of the primary key column in the DataFrame.
    - housenum_input_col (str): The name of the column in the DataFrame that provides the house number for the API.
    - street_input_col (str): The name of the column in the DataFrame that provides the street name for the API.
    - boro_input_col (str): The name of the column in the DataFrame that provides the borough for the API (required if zip is not given).
    - zip_input_col (str): The name of the column in the DataFrame that provides the zip code for the API (required if borough is not given).
    - response_columns (dict): Optional. A dictionary specifying which API response columns you want to keep.

    Returns:
    - pd.DataFrame: The merged DataFrame containing the original data and the filtered API response data.
    """

    # Create a session object
    session = requests.Session()
    session.headers.update(headers)

    # Define the function to send a request
    def send_request(house_number, street, borough=None, zip_code=None):
        params = {
            'houseNumber': house_number,
            'street': street,
        }
        if borough:
            params['borough'] = borough
        if zip_code:
            params['zip'] = zip_code

        try:
            response = session.get(api_endpoint, params=params, headers=headers)
            if response.status_code == 200:
                json_response = response.json()  # Parse the JSON response
                if 'address' in json_response:
                    return json_response['address']  # Return the 'address' object
                else:
                    return {}
            else:
                return {}
        except Exception as e:
            return {}

    # Prepare data for processing
    house_numbers = df_name[housenum_input_col].tolist()
    streets = df_name[street_input_col].tolist()
    boroughs = df_name[boro_input_col].tolist() if boro_input_col else [None] * len(df_name)
    zip_codes = df_name[zip_input_col].tolist() if zip_input_col else [None] * len(df_name)
    key_field_values = df_name[df_key_field].tolist()

    # List to store results
    results = []

    # Calculate the delay needed to stay within the rate limit
    delay_per_request = 60 / 2500  # 60 seconds divided by 2500 requests

    # Send requests sequentially with delay
    for house_number, street, borough, zip_code in zip(house_numbers, streets, boroughs, zip_codes):
        result = send_request(house_number, street, borough, zip_code)
        results.append(result)
        time.sleep(delay_per_request)  # Delay between requests to respect the rate limit

    # Convert the list of responses to a DataFrame
    if results and any(results):  # Check if results list is not empty and contains non-empty dictionaries
        response_df = pd.DataFrame(results)

        # If response_columns dictionary is provided, filter to keep only those columns
        if response_columns:
            response_df = response_df[response_columns]

        # Add the df_key_field from the original dataframe to the response_df for merging
        response_df[df_key_field] = key_field_values

        # Perform a left join of the original DataFrame with the response DataFrame on df_key_field
        merged_df = pd.merge(df_name, response_df, on=df_key_field, how='left')
    else:
        # If all results are empty, return the original DataFrame
        merged_df = df_name.copy()

    # Close the session when done
    session.close()

    return merged_df

In [ ]:
# Create a copy of the nyc address pandas dataframe and sample 100 records

address_input_df = nyc_address_df.sample(n=100, random_state=1).copy()

In [ ]:
# Prepare parameters for API

# Read the subscription key from a text file

with open('C:/.../OTI geoclient API primary key.txt', 'r') as file:
    subscription_key = file.read().strip()

# Set the headers with the subscription key
headers_param = {
    'Cache-Control': 'no-cache',
    'Ocp-Apim-Subscription-Key': subscription_key
}

address_api_url_param = "https://api.nyc.gov/geoclient/v2/address"

search_return_columns_to_keep = ['bbl', 'bblBoroughCode', 'bblTaxBlock',
    'bblTaxLot', 'buildingIdentificationNumber', 'latitude', 'longitude',
    'xCoordinate', 'yCoordinate', 'communityDistrict', 'communityDistrictNumber',
    'geosupportFunctionCode',
    'geosupportReturnCode', 'geosupportReturnCode2', 'returnCode1a', 'returnCode1e'
]

In [26]:
# Call the API using the custom function

oti_api_address_output_df = oti_geoclient_api_v2_address_endpoint(
    api_endpoint= address_api_url_param,
    headers= headers_param,
    df_name= address_input_df,
    df_key_field='row_id',
    housenum_input_col = 'house_number', street_input_col = 'street_name' , boro_input_col= 'borough' , zip_input_col= 'postcode',
    response_columns= search_return_columns_to_keep)

In [27]:
# Review api output dataframe

oti_api_address_output_df.head(100)

,row_id,house_number,street_name,borough,postcode,bbl,bblBoroughCode,bblTaxBlock,bblTaxLot,buildingIdentificationNumber,...,longitude,xCoordinate,yCoordinate,communityDistrict,communityDistrictNumber,geosupportFunctionCode,geosupportReturnCode,geosupportReturnCode2,returnCode1a,returnCode1e
0,2597,141,5 AVENUE,Brooklyn,11217,3009470011,3,00947,0011,3019401,...,-73.979230,0990011,0186368,306,06,1B,00,00,00,00
1,4698,305,EAST HOUSTON STREET,Manhattan,10002,1003500056,1,00350,0056,1004268,...,-73.983445,0988839,0202084,103,03,1B,00,00,00,00
2,3978,411,EAST 10 STREET,Manhattan,10009,1003820100,1,00382,0100,1078024,...,-73.976910,0990650,0203643,103,03,1B,00,00,00,00
3,2237,1597,NEW YORK AVENUE,Brooklyn,11210,3075610037,3,07561,0037,3428759,...,-73.944801,0999571,0170098,317,17,1B,00,00,00,00
4,2295,511,EAST 20 STREET,Manhattan,10010,1009780001,1,00978,0001,1083689,...,-73.977340,0990530,0206629,106,06,1B,00,01,01,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2618,1985,JEROME AVENUE,Bronx,10453,2028630050,2,02863,0050,2128676,...,-73.908718,1009503,0249821,205,05,1B,00,00,00,00
96,546,311,WEST 114 STREET,Manhattan,10026,1018480007,1,01848,0007,1055828,...,-73.956746,0996225,0231946,110,10,1B,00,00,00,00
97,3556,2474,LINDEN BOULEVARD,Brooklyn,11208,3045040004,3,04504,0004,3098845,...,-73.872457,1019633,0182084,305,05,1B,00,00,00,00
98,4547,79,STANHOPE STREET,Brooklyn,11221,3032540057,3,03254,0057,3424457,...,-73.924582,1005163,0193036,304,04,1B,00,00,00,00


In [ ]:
# Export geocoded output to csv

oti_api_address_output_df.to_csv('oti_api_address_output_df.csv', index=False)

**Example 2: Using the OTI Geoclient BIN API endpoint**

A BIN (Building Identification Nummber) is a unique, immutable, citywide standard for building identification developed by NYC Department of City Planning. It is a 7-byte numeric item. You can read more about them here: https://nycplanning.github.io/Geosupport-UPG/chapters/chapterVI/section03/

In [15]:
# Create a custom function to make API calls to the 'BIN' endpoint

def oti_geoclient_api_v2_bin_endpoint(api_endpoint, headers, df_name, df_key_field, api_input_column, response_columns=None):
    """
    Fetch data from the OTI geoclient API, merge the response with the original dataframe, and return the merged dataframe.

    Parameters:
    - api_endpoint (str): The API endpoint URL.
    - headers (dict): The headers to send with the API request.
    - df_name (pd.DataFrame): The input pandas DataFrame.
    - df_key_field (str): The name of the primary key column in the DataFrame.
    - api_input_column (str): The name of the column in the DataFrame that provides input for the API.
    - response_columns (dict): Optional. A dictionary specifying which API response columns you want to keep.

    Returns:
    - pd.DataFrame: The merged DataFrame containing the original data and the filtered API response data.
    """

    # Create a session object
    session = requests.Session()
    session.headers.update(headers)

    # Define the function to send a request
    def send_request(bin_input):
        params = {'bin': bin_input}
        #print(f"Sending request to API with URL: {api_endpoint} and headers: {headers}")  # Print the full URL and headers
        try:
            response = session.get(api_endpoint, params=params, headers=headers)
            if response.status_code == 200:
                json_response = response.json()  # Parse the JSON response
                if 'bin' in json_response:
                    return json_response['bin']  # Return the 'bin' object
                else:
                    return {}
            else:
                return {}
        except Exception as e:
            #print(f"Request failed for {bin_input}: {e}")
            return {}

    # Prepare data for processing
    bins = df_name[api_input_column].tolist()
    key_field_values = df_name[df_key_field].tolist()

    # List to store results
    results = []

    # Calculate the delay needed to stay within the rate limit
    delay_per_request = 60 / 2500  # 60 seconds divided by 2500 requests

    # Send requests sequentially with delay
    for bin_input in bins:
        result = send_request(bin_input)
        results.append(result)
        time.sleep(delay_per_request)  # Delay between requests to respect the rate limit

    # Convert the list of responses to a DataFrame
    if results and any(results):  # Check if results list is not empty and contains non-empty dictionaries
        response_df = pd.DataFrame(results)

        # If response_columns dictionary is provided, filter to keep only those columns
        if response_columns:
            response_df = response_df[response_columns]

        # Add the df_key_field from the original dataframe to the response_df for merging
        response_df[df_key_field] = key_field_values

        # Perform a left join of the original DataFrame with the response DataFrame on df_key_field
        merged_df = pd.merge(df_name, response_df, on=df_key_field, how='left')
    else:
        # If all results are empty, return the original DataFrame
        #print("API returned empty results for all rows.")
        merged_df = df_name.copy()

    # Close the session when done
    session.close()

    return merged_df

In [28]:
# Prepare a pandas dataframe to pass to OTI api

# Specify the columns you want to keep from the original dataframe

input_columns_to_keep = ['row_id', 'buildingIdentificationNumber']  # Replace with the columns you want to keep

# Create a copy of the OTI address output pandas dataframe and sample 50 records

bin_input_df = oti_api_address_output_df[oti_api_address_output_df['buildingIdentificationNumber'].notna()][input_columns_to_keep].sample(n=50, random_state=1).copy()

In [29]:
# Prepare parameters for API

# Note the headers_param was already set in Example #1

bin_api_url_param = "https://api.nyc.gov/geoclient/v2/bin"

bin_return_columns_to_keep = ['bbl', 'bblBoroughCode', 'bblTaxBlock',
    'bblTaxLot',
    'internalLabelXCoordinate', 'internalLabelYCoordinate',
    'geosupportFunctionCode',
    'geosupportReturnCode'
]

In [30]:
# Call the API using the custom function

oti_api_bin_output_df = oti_geoclient_api_v2_bin_endpoint(
    api_endpoint= bin_api_url_param,
    headers= headers_param,
    df_name= bin_input_df,
    df_key_field='row_id',
    api_input_column='buildingIdentificationNumber',
    response_columns= bin_return_columns_to_keep)

In [31]:
# Review api output dataframe

oti_api_bin_output_df.head(10)

,row_id,buildingIdentificationNumber,bbl,bblBoroughCode,bblTaxBlock,bblTaxLot,internalLabelXCoordinate,internalLabelYCoordinate,geosupportFunctionCode,geosupportReturnCode
0,5409,3421892,3044527506,3,04452,7506,NaN,NaN,BN,00
1,2160,1060383,1020250053,1,02025,0053,0999928,0237684,BN,00
2,793,3029892,3012030066,3,01203,0066,0999877,0186138,BN,00
3,2460,3428849,3011330032,3,01133,0032,0995374,0186683,BN,00
4,201,3048978,3017620069,3,01762,0069,0996785,0191154,BN,00
5,4734,1091016,1002530001,1,00253,0001,0985518,0198288,BN,00
6,3397,1060822,1020450057,1,02045,0057,1000554,0239762,BN,00
7,2782,2092211,2030160071,2,03016,0071,1017062,0244994,BN,00
8,3493,2023821,2037390023,2,03739,0023,1017593,0240683,BN,00
9,367,2004753,2026630031,2,02663,0031,1011891,0242010,BN,00


In [ ]:
# Export geocoded output to csv

oti_api_bin_output_df.to_csv('oti_api_bin_output_df.csv', index=False)

**Example 3: Calling the OTI Geoclient BBL API endpoint**

A Borough-Block-and-Lot (BBL) is a single data item used that can be used to uniquely identify a city tax lot. It is maintained by the NYC Department of Finance (DOF). A city tax lot is a a subdivision of the broader city tax geography, which DOF manages.

You can read more about them here: https://nycplanning.github.io/Geosupport-UPG/chapters/chapterVI/section02/

In [32]:
# Create a custom function to make API calls to the 'BBL' endpoint

def oti_geoclient_api_v2_bbl_endpoint(api_endpoint, headers, df_name, df_key_field, boro_input_col, block_input_col, lot_input_col, response_columns=None):
    """
    Fetch data from the OTI geoclient API, merge the response with the original dataframe, and return the merged dataframe.

    Parameters:
    - api_endpoint (str): The API endpoint URL.
    - headers (dict): The headers to send with the API request.
    - df_name (pd.DataFrame): The input pandas DataFrame.
    - df_key_field (str): The name of the primary key column in the DataFrame.
    - boro_input_col (str): The name of the column in the DataFrame that provides the borough input for the API.
    - block_input_col (str): The name of the column in the DataFrame that provides the block input for the API.
    - lot_input_col (str): The name of the column in the DataFrame that provides the lot input for the API.
    - response_columns (dict): Optional. A dictionary specifying which API response columns you want to keep.

    Returns:
    - pd.DataFrame: The merged DataFrame containing the original data and the filtered API response data.
    """

    # Create a session object
    session = requests.Session()
    session.headers.update(headers)

    # Define the function to send a request
    def send_request(borough, block, lot):
        params = {
            'borough': borough,
            'block': block,
            'lot': lot
        }
        #print(f"Sending request to API with URL: {api_endpoint}, params: {params}, and headers: {headers}")  # Print the full URL, params, and headers
        try:
            response = session.get(api_endpoint, params=params)
            if response.status_code == 200:
                json_response = response.json()  # Parse the JSON response
                if 'bbl' in json_response:
                    return json_response['bbl']  # Return the 'bbl' object
                else:
                    return {}
            else:
                return {}
        except Exception as e:
            #print(f"Request failed for bbl {borough}{block}{lot}: {e}")
            return {}

    # Prepare data for processing
    boroughs = df_name[boro_input_col].tolist()
    blocks = df_name[block_input_col].tolist()
    lots = df_name[lot_input_col].tolist()
    key_field_values = df_name[df_key_field].tolist()

    # List to store results
    results = []

    # Calculate the delay needed to stay within the rate limit
    delay_per_request = 60 / 2500  # 60 seconds divided by 2500 requests

    # Send requests sequentially with delay
    for borough, block, lot in zip(boroughs, blocks, lots):
        result = send_request(borough, block, lot)
        results.append(result)
        time.sleep(delay_per_request)  # Delay between requests to respect the rate limit

    # Convert the list of responses to a DataFrame
    if results and any(results):  # Check if results list is not empty and contains non-empty dictionaries
        response_df = pd.DataFrame(results)

        # If response_columns dictionary is provided, filter to keep only those columns
        if response_columns:
            response_df = response_df[response_columns]

        # Add the df_key_field from the original dataframe to the response_df for merging
        response_df[df_key_field] = key_field_values

        # Perform a left join of the original DataFrame with the response DataFrame on df_key_field
        merged_df = pd.merge(df_name, response_df, on=df_key_field, how='left')
    else:
        # If all results are empty, return the original DataFrame
        #print("API returned empty results for all rows.")
        merged_df = df_name.copy()

    # Close the session when done
    session.close()

    return merged_df

In [ ]:
# Prepare a pandas dataframe to pass to OTI api

# Specify the columns you want to keep from the original dataframe

input_columns_to_keep = ['row_id', 'bblBoroughCode', 'bblTaxBlock', 'bblTaxLot']  # Replace with the columns you want to keep

# Create a copy of the OTI address output pandas dataframe and sample 50 records

bbl_input_df = oti_api_address_output_df[oti_api_address_output_df['bblBoroughCode'].notna()][input_columns_to_keep].sample(n=50, random_state=1).copy()

In [35]:
# Prepare parameters for API

# Note the headers_param was already set in Example #1

bbl_api_url_param = "https://api.nyc.gov/geoclient/v2/bbl"

bbl_return_columns_to_keep = ['bbl','buildingIdentificationNumber',
    'latitudeInternalLabel','longitudeInternalLabel',
    'internalLabelXCoordinate', 'internalLabelYCoordinate',
    'numberOfEntriesInListOfGeographicIdentifiers','numberOfExistingStructuresOnLot',
    'numberOfStreetFrontagesOfLot',
    'geosupportFunctionCode',
    'geosupportReturnCode', 'returnCode1a'
]

In [36]:
# Call the API using the custom function

oti_api_bbl_output_df = oti_geoclient_api_v2_bbl_endpoint(
    api_endpoint= bbl_api_url_param,
    headers= headers_param,
    df_name= bbl_input_df,
    df_key_field='row_id',
    boro_input_col='bblBoroughCode',
    block_input_col='bblTaxBlock',
    lot_input_col='bblTaxLot',
    response_columns= bbl_return_columns_to_keep)

In [37]:
# Review api output dataframe

oti_api_bbl_output_df.head(10)

,row_id,bblBoroughCode,bblTaxBlock,bblTaxLot,bbl,buildingIdentificationNumber,latitudeInternalLabel,longitudeInternalLabel,internalLabelXCoordinate,internalLabelYCoordinate,numberOfEntriesInListOfGeographicIdentifiers,numberOfExistingStructuresOnLot,numberOfStreetFrontagesOfLot,geosupportFunctionCode,geosupportReturnCode,returnCode1a
0,5409,3,04452,7506,3044527506,3421884,NaN,NaN,NaN,NaN,0009,0009,01,BL,00,00
1,2160,1,02025,0053,1020250053,1060383,40.819050,-73.943357,0999928,0237684,0001,0001,01,BL,00,00
2,793,3,01203,0066,3012030066,3029892,40.677569,-73.943661,0999877,0186138,0001,0001,01,BL,00,00
3,2460,3,01133,0032,3011330032,3000000,40.679072,-73.959894,0995374,0186683,0003,0001,01,BL,00,00
4,201,3,01762,0069,3017620069,3048978,40.691342,-73.954799,0996785,0191154,0001,0001,01,BL,00,00
5,4734,1,00253,0001,1002530001,1077585,40.710932,-73.995426,0985518,0198288,0021,0014,04,BL,01,01
6,3397,1,02045,0057,1020450057,1060822,40.824752,-73.941090,1000554,0239762,0001,0001,01,BL,00,00
7,2782,2,03016,0071,2030160071,2092211,40.839067,-73.881418,1017062,0244994,0001,0001,01,BL,00,00
8,3493,2,03739,0023,2037390023,2023821,40.827232,-73.879520,1017593,0240683,0001,0001,01,BL,00,00
9,367,2,02663,0031,2026630031,2004753,40.830894,-73.900118,1011891,0242010,0002,0001,02,BL,00,00


In [ ]:
# Export geocoded output to csv

oti_api_bbl_output_df.to_csv('oti_api_bbl_output_df.csv', index=False)